In [25]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
wallet.import_key('test', '5JMXaLz5xnVvwrnvAGaZKQZFCDdeU6wjmuJY1rDnXiUZz7Gyi1o')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

done!


In [ ]:
eosapi.get_public_key('5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

In [ ]:
eosapi.get_public_key('5JMXaLz5xnVvwrnvAGaZKQZFCDdeU6wjmuJY1rDnXiUZz7Gyi1o')

In [ ]:
eosapi.get_balance('helloworld11')

In [ ]:
key1 = 'EOS7ZC3YFTCpPknEniq5xiHxHBt2tSW2E5eMXEcxHDD54UBWEgN4G'
key2 = 'EOS7ZC3YFTCpPknEniq5xiHxHBt2tSW2E5eMXEcxHDD54UBWEgN4G'
eosapi.create_account('helloworld11', 'helloworld33', key1, key2, 64*1024, 1.0, 1.0)

In [28]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [60]:
from pyeoskit import db
db.reset()
code = r'''
#include <eosio/eosio.hpp>
#include <eosio/action.hpp>
#include <eosio/print.hpp>

extern "C" {
    __attribute__((eosio_wasm_import))
    int call_contract_get_extra_args(void* extra_args, size_t size1);

    __attribute__((eosio_wasm_import))
    int call_contract_set_results(void* result, size_t size1);

    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        eosio::print("hello,world\n");
    }

   void call(uint64_t func, uint64_t arg1, uint64_t arg2, uint64_t arg3) {
      if (func == eosio::name("calltest1").value) {
         uint64_t extra_args;
         ::call_contract_get_extra_args(&extra_args, sizeof(extra_args));
         eosio::print("+++++++++++call: extra_args:", extra_args, "\n");
         extra_args += 1;
         ::call_contract_set_results(&extra_args, sizeof(extra_args));
      }
   }
}
'''

r = open('test.cpp', 'w').write(code)


%system rm test.obj test.wasm
%system eosio-cpp -I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/capi -I/usr/local/Cellar/eosio.cdt/1.6.1/opt/eosio.cdt/include/eosiolib/core -O3 -contract test -o test.obj -c test.cpp
%system eosio-ld test.obj -o test.wasm


account_name = 'helloworld33'
code = open('test.wasm', 'rb').read()
m = hashlib.sha256()
m.update(code)
code_hash = m.hexdigest()
r = eosapi.get_code(account_name)
if code_hash != r['code_hash']:
    abi = ''
    print('update code')
    r = eosapi.set_contract(account_name, code, abi, 0)


[]

['Warning, empty ricardian clause file',
 'Warning, empty ricardian clause file']

[]

In [62]:
from pyeoskit import db
db.reset()
code = '''
def apply(receiver, code, action):
    ret = call_contract('helloworld33', 'calltest1', 0, 0, 0, int.to_bytes(1, 8, 'little'))
    print(ret)
    print(int.from_bytes(ret, 'little'))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

True

b'\x02\x00\x00\x00\x00\x00\x00\x00'
2

